In [1]:
import json 
import re
import numpy as np
from astropy.table import Table
from astropy.io import ascii

result_list = []
with open("../data/visual_check_catalog/peak_check_v4.json","r") as f:
    for line in f:
        data_tag = json.loads(line)
        result_list.append(data_tag)

tag_list = []
note_list = []
name_list = []
for item in result_list:
    tags = item['annotation']['labels']
    tag = tags[0]
    note = item['annotation']['note']
    name = item['content']
    name = re.search(r"___\d_(.*)",name).group(1)
    if len(tags)>1:
        print(tags,'---',name)
    tag_list.append(tag)
    if len(note) == 0: note = '-'
    note_list.append(note)
    name_list.append(name)
#print(re.search(r"___\d_(.*)",name).group(1))
#https://www.runoob.com/python/python-reg-expressions.html
tag_list = np.array(tag_list)
note_list = np.array(note_list)
name_list = np.array(name_list)
    
#write peak catalog,use astropy table func
id_single = np.where(tag_list=='single')
name_single = np.copy(name_list[id_single])
note_single = np.copy(name_list[id_single])

id_double = np.where(tag_list=='double')
name_double = np.copy(name_list[id_double])
note_double = np.copy(name_list[id_double])

id_SD_peak = np.where(tag_list=='S_or_D')
name_SD_peak = np.copy(name_list[id_SD_peak])
note_SD_peak = np.copy(name_list[id_SD_peak])

id_all = np.concatenate((id_single[0],id_double[0],id_SD_peak[0]))
name_all = name_list[id_all]
tag_all = tag_list[id_all]
note_all = note_list[id_all]


plate_all = []
fiber_all = []
mjd_all = []
for item in name_all:
    tmp = item.split('-')
    plate_all.append(tmp[0])
    mjd_all.append(tmp[1])
    fiber_all.append(tmp[2].split('_')[0])

t = Table([id_all, plate_all, mjd_all, fiber_all, tag_all, note_all], names=('id@json', 'plate', 'mjd', 'fiber', 'tag', 'note'), meta={'name': 'first table'})
#t.write('../data/candidate_list_for_idl_1.txt',format='ascii.csv',overwrite=True)

In [3]:
t

id@json,plate,mjd,fiber,tag,note
int64,unicode5,unicode5,unicode3,unicode8,unicode35
2,10243,58159,381,single,-
3,10272,58172,937,single,-
4,10274,58170,211,single,-
5,10287,58101,813,single,-
6,10429,58145,395,single,-
7,10431,58137,159,single,-
9,10435,58156,475,single,-
10,10441,58138,465,single,-
11,10446,58133,607,single,-


In [4]:
import h5py
base_dir = '../data/2/'
id_no_qso = []
for ii in range(len(plate_all)): #
    file_name = '{}_{}.h5'.format(plate_all[ii],mjd_all[ii])
    h5_file = base_dir+file_name
    with h5py.File(h5_file, 'r') as f:
        arr2 = f['zans']
        zans_arr = arr2[:]
    #print('file {} read finished'.format(file_name))
    fibers = np.array([item[3] for item in zans_arr],dtype=np.int)  #refer to below to find index, Note index-1 !!!
    id_fiber = np.where(fibers == int(fiber_all[ii]))
    
    #extract the structure which save the spectrum info
    struct = zans_arr[id_fiber][0]
    #save some info
    ra = struct[8]
    dec = struct[9]
    #print(ra,dec)\
    z_l = struct[12]
    vdisp = struct[22]
    #still need 
    #1. i-band magnitude
    #2. sn of peak/second peak
    #3. einstein radius
    #4. wavelength of peak 
    mag_i = struct 
    obj_class = struct[10] 
    
    if ii%100 == 1:
        print('read {} file'.format(ii))
        
    if obj_class == 'GALAXY':
        id_no_qso.append(ii)
    else:
        print(obj_class)
        print("{}-{}-{}".format(plate_all[ii], mjd_all[ii],fiber_all[ii]))
        #readspec, 3960, 195,mjd=55663,zans=zans,plug=plug
        
plate_nqso = [plate_all[idx] for idx in id_no_qso]
mjd_nqso = [mjd_all[idx] for idx in id_no_qso]
fiber_nqso = [fiber_all[idx] for idx in id_no_qso]
id_nqso = [id_all[idx] for idx in id_no_qso]
tag_nqso = [tag_all[idx] for idx in id_no_qso]
note_nqso = [note_all[idx] for idx in id_no_qso]

t = Table([id_nqso, plate_nqso, mjd_nqso, fiber_nqso, tag_nqso, note_nqso], names=('id@json', 'plate', 'mjd', 'fiber', 'tag', 'note'), meta={'name': 'first table'})
#t.write('../data/candidate_list_nqso.csv',format='ascii.csv',overwrite=True)

read 1 file
QSO   
3960-55663-195
QSO   
4085-55452-553
read 101 file
read 201 file
read 301 file
QSO   
6678-56401-745
read 401 file
QSO   
9237-58022-764
QSO   
9380-58081-403
read 501 file
QSO   
6028-56102-785
QSO   
8820-57449-989
QSO   
4441-55532-997
read 601 file
QSO   
9225-58013-402
read 701 file
QSO   
4768-55944-407


In [5]:
len(obj_class)

6

In [6]:
obj_class

'GALAXY'